# Implementação do Random Forest

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from pandas import DataFrame
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

TRAINING_DATASET_SOURCE = '../datasets/training_data.csv'  # Since we are one directory up, we should go down one directory to import the datasets
TEST_DATASET_SOURCE = '../datasets/test_data.csv'

train_df = pd.read_csv(TRAINING_DATASET_SOURCE)
test_df = pd.read_csv(TEST_DATASET_SOURCE)

# Definição dos dados de teste e de treino

In [10]:
print(train_df.shape, test_df.shape)
print(train_df['incidents'].value_counts())

incidents_count = train_df['incidents'].value_counts()

max_count = incidents_count.max()
print('Max value count:', max_count)

df_classes = []
for category, counts in zip(incidents_count.index, incidents_count):
    #print(category, counts)
    df_classes.append(train_df[train_df['incidents'] == category])

df_classes_over = []

for category in df_classes:
    df_classes_over.append(category.sample(max_count, replace=True))

df_test_over = pd.concat(df_classes_over, axis=0)

print(df_test_over['incidents'].value_counts())


(5000, 13) (1206, 12)
None         2028
High         1073
Low           718
Very_High     603
Medium        578
Name: incidents, dtype: int64
Max value count: 2028
None         2028
High         2028
Low          2028
Very_High    2028
Medium       2028
Name: incidents, dtype: int64


In [11]:
features = df_test_over.drop(['incidents'], axis=1)
target = df_test_over['incidents']

all_features = features.columns.tolist()

features[:5]

,city_name,magnitude_of_delay,delay_in_seconds,affected_roads,record_date,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_precipitation,avg_rain
1763,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-06-27 12:00,LIGHT,18.0,1017.0,69.0,4.0,0.0,Sem Chuva
2455,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-11-24 06:00,DARK,6.0,1016.0,79.0,1.0,0.0,Sem Chuva
2935,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-10-10 13:00,LIGHT,23.0,1019.0,25.0,4.0,0.0,Sem Chuva
4125,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-07-16 05:00,DARK,23.0,1016.0,40.0,3.0,0.0,Sem Chuva
3678,Guimaraes,UNDEFINED,0,",",2021-03-14 21:00,DARK,11.0,1028.0,84.0,1.0,0.0,Sem Chuva


### Obtenção das features numericas e categoricas

In [5]:
from sklearn.preprocessing import StandardScaler

categorical_to_numerical = {
    'luminosity': {
        'LOW_LIGHT': 0,
        'LIGHT': 1,
        'DARK': 2
    },
    'avg_rain': {
        'Sem Chuva': 0,
        'chuva fraca': 1,
        'chuva moderada': 2,
        'chuva forte': 3,
    }
}

def decision_tree_data_preparation(df: DataFrame) -> DataFrame:
    dropped_columns = ['city_name', 'avg_precipitation', 'magnitude_of_delay']

    numerical_features = [column for column, dtype in zip(features.columns, features.dtypes) if
                          dtype.kind in ['i', 'f'] and column not in dropped_columns]

    numerical_features = ['avg_temperature', 'avg_atm_pressure', 'avg_humidity', 'avg_wind_speed', 'luminosity']
    #assert numerical_features == ['avg_temperature', 'avg_atm_pressure', 'avg_humidity', 'avg_wind_speed', 'luminosity']


    categorical_features = [column for column, dtype in zip(features.columns, features.dtypes) if
                            dtype.kind not in ['i', 'f'] and column not in dropped_columns]

    prep_df = df.drop(dropped_columns, axis=1)
    prep_df.drop_duplicates()

    ### Converter as features categoricas em numericas
    prep_df.replace(categorical_to_numerical, inplace=True)

    ### Extrair a hora e dia da semana da feature 'record_date'
    record_date = pd.DatetimeIndex(prep_df['record_date'])

    prep_df['hour'] = record_date.hour
    prep_df['day'] = record_date.day
    prep_df['month'] = record_date.month
    prep_df['weekday'] = record_date.weekday

    prep_df.drop(columns=['record_date'], inplace=True)

    #train_df['affected_roads'] = train_df['affected_roads'].fillna(train_df['affected_roads'].mode().iloc[0])

    num_affected_roads = []
    for line in df['affected_roads']:
        unique_roads = set(str(line).split(','))
        valid_roads = [elem for elem in unique_roads if elem != '']
        count = len(valid_roads)
        num_affected_roads.append(count)

    prep_df['num_affected_roads'] = num_affected_roads

    ### Ao analisar o resultado pós-tratamento, verificámos que a feature 'affected_roads' tinha alta correlação com 'delay_in_minutes'
    prep_df.drop(columns=['affected_roads'], inplace=True)

    ### Converter a feature 'delay_in_seconds' para 'delay_in_minutes' de modo a reduzir o intervalo de valores
    delay_in_minutes = prep_df['delay_in_seconds'].map(lambda seconds : seconds / 60)

    prep_df.drop(columns=['delay_in_seconds'], inplace=True)
    prep_df['delay_in_minutes'] = delay_in_minutes

    ### Limites superior e inferior (sem outliers) dos diagramas de caixa
    #numerical_features.remove('delay_in_seconds')
    #numerical_features.append('delay_in_minutes')

    return prep_df

### Divisão dos dados em dados de teste e treino

In [12]:
target_num = target.map({'None': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very_High': 4})
target_num

1763    0
2455    0
2935    0
4125    0
3678    0
       ..
3556    2
3505    2
2507    2
171     2
4483    2
Name: incidents, Length: 10140, dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(decision_tree_data_preparation(features), target_num, test_size=0.3,
                                                    random_state=2000)

## Hyperparameter Tuning

In [302]:
import numpy as np

n_estimators = [100, 200] #[int(x) for x in np.linspace(start=10, stop=500, num=5)]

max_features = [int(x) for x in np.linspace(start=6, stop=12, num=3)]

max_depth = [int(x) for x in np.linspace(start=10, stop=500, num=5)]

min_samples_split = [int(x) for x in np.linspace(start=2, stop=30, num=2)]  #[2, 5, 10, 20]

min_samples_leaf = [int(x) for x in np.linspace(start=2, stop=30, num=2)]  # [1, 2, 5, 10]

max_leaf_nodes = [int(x) for x in np.linspace(start=20, stop=200, num=5)]

bootstrap = [True, False]

criterions = ["gini", "entropy", "log_loss"]

### Obtenção das previsões do dataset de submissão

In [309]:
test_data = test_df.copy()

test_data_prepared = decision_tree_data_preparation(test_data)

predictions = rf_RandomGrid.predict(test_data_prepared) #RF_Model.predict(test_data_prepared)
predictions_df = pd.DataFrame(predictions)
predictions_df.index += 1
predictions_df.to_csv("../submission_v2.csv", header=['Incidents'], index_label='RowId')